In [2]:
# Load the Drive helper and mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir("drive/My Drive/SRGAN_project")
!ls

 benchmark_results   __pycache__	   sr_images	      testSRGAN.ipynb
 dataset	     pytorch_ssim	   statistics
 models		     runSRGAN.ipynb	   test_images
 networks.py	     SRGAN_project.ipynb  'test_images (1)'


In [4]:
# Imports

import torch
import math
import os
from os import listdir
from os.path import join
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
from torchvision.transforms import Compose, RandomCrop, ToTensor, Normalize, ToPILImage, CenterCrop, Resize, Grayscale
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm
from os.path import join
from torch import nn, optim
from torchvision.models.vgg import vgg16
from torchvision.utils import save_image
from IPython.display import Image as IM

from networks import *

In [5]:
# Checking for device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# Hyperparameters

UPSCALE_FACTOR = 4
CROP_SIZE = 88
N_EPOCHS = 150
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [7]:
# Dataset Utility Functions

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)


def train_hr_transform(crop_size):
    return Compose([
        RandomCrop(crop_size),
        ToTensor(),
    ])

# def just_hr_transform(crop_size):
#     return Compose([
#         RandomCrop(crop_size),
#         ToTensor(),
#     ])


def train_lr_transform(crop_size, upscale_factor):
    return Compose([
        ToPILImage(),
        Resize(crop_size // upscale_factor, interpolation = Image.BICUBIC),
        ToTensor()
    ])


def display_transform():
    return Compose([
        ToPILImage(),
        Resize(400),
        CenterCrop(400),
        ToTensor()
    ])
  

class TrainDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, crop_size, upscale_factor):
        super(TrainDatasetFromFolder, self).__init__()
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]
        crop_size = calculate_valid_crop_size(crop_size, upscale_factor)
        self.hr_transform = train_hr_transform(crop_size)
        self.lr_transform = train_lr_transform(crop_size, upscale_factor)

    def __getitem__(self, index):
        hr_image = self.hr_transform(Image.open(self.image_filenames[index]))
        lr_image = self.lr_transform(hr_image)
        return lr_image, hr_image

    def __len__(self):
        return len(self.image_filenames)

class ValDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(ValDatasetFromFolder, self).__init__()
        self.upscale_factor = upscale_factor
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

    def __getitem__(self, index):
        hr_image = Image.open(self.image_filenames[index]).convert('RGB')
        w, h = hr_image.size
        crop_size = calculate_valid_crop_size(min(w, h), self.upscale_factor)
        lr_scale = Resize(crop_size // self.upscale_factor, interpolation=Image.BICUBIC)
        hr_scale = Resize(crop_size, interpolation=Image.BICUBIC)
        hr_image = CenterCrop(crop_size)(hr_image)
        lr_image = lr_scale(hr_image)
        hr_restore_img = hr_scale(lr_image)
        return ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.image_filenames)

class TestDatasetFromFolder(Dataset):
    def __init__(self, dataset_dir, upscale_factor):
        super(TestDatasetFromFolder, self).__init__()
        self.upscale_factor = upscale_factor
        self.image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir) if is_image_file(x)]

    def __getitem__(self, index):
        image_name = self.image_filenames[index].split('/')[-1]
        hr_image = Image.open(self.image_filenames[index]).convert('RGB')
        w, h = hr_image.size
        crop_size = calculate_valid_crop_size(min(w, h), self.upscale_factor)
        lr_scale = Resize(crop_size // self.upscale_factor, interpolation=Image.BICUBIC)
        hr_scale = Resize(crop_size, interpolation=Image.BICUBIC)
        hr_image = CenterCrop(crop_size)(hr_image)
        lr_image = lr_scale(hr_image)
        hr_restore_img = hr_scale(lr_image)
        return str(image_name[0]), ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

    def __len__(self):
        return len(self.image_filenames)

# class TestDatasetFromFolder(Dataset):
#     def __init__(self, dataset_dir, upscale_factor):
#         super(TestDatasetFromFolder, self).__init__()
#         self.lr_path = dataset_dir + '/SRF_' + str(upscale_factor) + '/data/'
#         self.hr_path = dataset_dir + '/SRF_' + str(upscale_factor) + '/target/'
#         self.upscale_factor = upscale_factor
#         self.lr_filenames = [join(self.lr_path, x) for x in listdir(self.lr_path) if is_image_file(x)]
#         self.hr_filenames = [join(self.hr_path, x) for x in listdir(self.hr_path) if is_image_file(x)]

#     def __getitem__(self, index):
#         image_name = self.lr_filenames[index].split('/')[-1]
#         lr_image = Image.open(self.lr_filenames[index])
#         w, h = lr_image.size
#         hr_image = Image.open(self.hr_filenames[index])
#         hr_scale = Resize((self.upscale_factor * h, self.upscale_factor * w), interpolation=Image.BICUBIC)
#         hr_restore_img = hr_scale(lr_image)
#         return image_name, ToTensor()(lr_image), ToTensor()(hr_restore_img), ToTensor()(hr_image)

#     def __len__(self):
#         return len(self.lr_filenames)

In [8]:
# Loading GAN models

loadedG = torch.load('models/model01/Generator.model')
loadedD = torch.load('models/model01/Discriminator.model')

In [9]:
import pandas as pd

results = {'Set': {'psnr': [], 'ssim': []}, 'Set5': {'psnr': [], 'ssim': []}, 'Set14': {'psnr': [], 'ssim': []}, 'BSD100': {'psnr': [], 'ssim': []}}

# test_model = Generator(UPSCALE_FACTOR).eval()
# test_model.load_state_dict(torch.load('epochs/netG_epoch_4_134.pth'))
test_set_5 = TestDatasetFromFolder('dataset/benchmark/Set5/SRF_4/target/', upscale_factor=UPSCALE_FACTOR)
test_loader_5 = DataLoader(dataset=test_set_5, num_workers=4, batch_size=1, shuffle=False)
test_set_14 = TestDatasetFromFolder('dataset/benchmark/Set14/SRF_4/target/', upscale_factor=UPSCALE_FACTOR)
test_loader_14 = DataLoader(dataset=test_set_14, num_workers=4, batch_size=1, shuffle=False)
test_set_100 = TestDatasetFromFolder('dataset/benchmark/BSDS100/SRF_4/target/', upscale_factor=UPSCALE_FACTOR)
test_loader_100 = DataLoader(dataset=test_set_100, num_workers=4, batch_size=1, shuffle=False)
test_set = TestDatasetFromFolder('test_images/', upscale_factor=UPSCALE_FACTOR)
test_loader = DataLoader(dataset=test_set, num_workers=4, batch_size=1, shuffle=False)
# test_bar = tqdm(test_loader, desc='[testing benchmark datasets]')
test_bar = tqdm(test_loader)
test_bar_5 = tqdm(test_loader_5)
test_bar_14 = tqdm(test_loader_14)
test_bar_100 = tqdm(test_loader_100)

out_path = 'benchmark_results/SRF_' + str(UPSCALE_FACTOR) + '/'
out_path_5 = 'benchmark_results/SRF5_' + str(UPSCALE_FACTOR) + '/'
out_path_14 = 'benchmark_results/SRF14_' + str(UPSCALE_FACTOR) + '/'
out_path_100 = 'benchmark_results/SRF100_' + str(UPSCALE_FACTOR) + '/'
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path_5):
    os.makedirs(out_path_5)
if not os.path.exists(out_path_14):
    os.makedirs(out_path_14)
if not os.path.exists(out_path_100):
    os.makedirs(out_path_100)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]


  0%|          | 0/100 [00:00<?, ?it/s]